In [1]:
import numpy as np
import py3Dmol as p3d
import veloxchem as vlx

In [2]:
ethene_xyz = """6

C        0.67759997    0.00000000    0.00000000
C       -0.67759997    0.00000000    0.00000000
H        1.21655197    0.92414474    0.00000000
H        1.21655197   -0.92414474    0.00000000
H       -1.21655197   -0.92414474    0.00000000
H       -1.21655197    0.92414474    0.00000000
"""

In [3]:
viewer = p3d.view(width=400, height=300)
viewer.addModel(ethene_xyz)
viewer.setViewStyle({"style": "outline", "width": 0.05})
viewer.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
molecule = vlx.Molecule.from_xyz_string(ethene_xyz)
basis = vlx.MolecularBasis.read(molecule, "6-31g", ostream=None)

In [5]:
scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

scf_drv.xcfun = "b3lyp"

scf_results = scf_drv.compute(molecule, basis)

In [6]:
norb = basis.get_dimension_of_basis()
nocc = molecule.number_of_alpha_electrons()
nvirt = norb - nocc

n = nocc * nvirt

print("Number of occupied orbitals:", nocc)
print("Number of unoccupied orbitals:", nvirt)
print("Number of excitations:", n)

Number of occupied orbitals: 8
Number of unoccupied orbitals: 18
Number of excitations: 144


In [7]:
lres_drv = vlx.LinearResponseEigenSolver()
lres_drv.ostream.mute()

_ = lres_drv.compute(molecule, basis, scf_results)
E2 = lres_drv.get_e2(molecule, basis, scf_results)

A = E2[:n, :n]

In [8]:
eigs, X = np.linalg.eigh(A)

print("Excitation energies (eV):\n", eigs[:5] * 27.2114)

Excitation energies (eV):
 [8.57868869 9.00818369 9.22485575 9.60189688 9.76762482]


In [9]:
tda_drv = vlx.TdaEigenSolver()
tda_drv.ostream.mute()

tda_drv.nstates = 5

tda_results = tda_drv.compute(molecule, basis, scf_results)

In [10]:
print("Excitation energies (eV):\n", tda_results["eigenvalues"] * 27.2114)

Excitation energies (eV):
 [8.57868811 9.00818379 9.22485652 9.60189672 9.76762549]
